In [ ]:
# pip install git+https://github.com/lsst-sims/rubin_nights.git
import os
import numpy as np
from astropy.time import Time

from rubin_nights.connections import get_clients, get_access_token
from rubin_nights.reference_values import API_ENDPOINTS
from rubin_nights.consdb_query import ConsDbTap, ConsDbFastAPI, ConsDbSql
from rubin_nights.augment_visits import augment_visits, exclude_visits, fetch_excluded_visits
from rubin_nights.targets_and_visits import targets_and_visits 

## Query the ConsDB 

First, set up the location and authentication for the ConsDB.

If you are on an RSP - you're good to go, this will be taken care of automatically for you.
If you are outside an RSP, choose which site you'd like to use and identify your RSP token. 

In [ ]:
on_rsp = False
# Are you on an RSP?
if on_rsp:
    api_base = os.getenv("EXTERNAL_INSTANCE_URL", "")
    token = get_access_token()
    site = None
# Or are you outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    site = "usdf"
    api_base = API_ENDPOINTS[site]
    # Substitute the location of your own tokenfile
    # If you prefer, this will also get token info from an "ACCESS_TOKEN" environment variable
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
    token = get_access_token(tokenfile)

    
consdb_tap = ConsDbTap(api_base=api_base, token=token)
consdb_fastapi = ConsDbFastAPI(api_base=api_base, auth=('user', token))

# The ConsDbSql access will only work within the USDF or Summit, but connects using postgres directly 
# and then runs queries through pandas. 
if on_rsp:
    consdb_sql = ConsDbSql(site="usdf")

It's worth noting that the queries below are much faster when run at the RSP. Consdb_sql will be faster than either the FastAPI or TAP.

General queries are supported and the results can be manipulated as dataframes. See [sdm-schemas.lsst.io](https://sdm-schemas.lsst.io) for more information on available data.

In [ ]:
# Here is a simple query
query = "select * from cdb_lsstcam.visit1 where science_program = 'BLOCK-407' and day_obs=20251127"

Both the FastAPI and TAP interfaces offer options for sync or async queries.

In [ ]:
%%time
# sync tap query (uses equivalent of run_async)
visits_tap = consdb_tap.query(query)
print(len(visits_tap))
print(visits_tap.columns)

In [ ]:
%%time
# sync fastapi query (uses a synchronous httpx client)
visits = consdb_fastapi.query(query)
print(len(visits))
print(visits.columns)

In [ ]:
np.all(visits['visit_id'] == visits_tap['visit_id'])

The TAP client can be used in an async manner by accessing the TAPService directly - `consdb_tap.tap` : 

In [ ]:
job = consdb_tap.tap.submit_job(query)
job.run()
## Either set the timeout to the max time you want to wait, or enter a loop until the phase matches one of the below values
print(job.phase)
job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=120)
print(job.phase)
results = job.fetch_result().to_table().to_pandas()
results.head(3)[['visit_id', 'obs_start', 's_ra', 's_dec', 'sky_rotation', 'band', 'target_name', 'observation_reason']]

The FastAPI client can be used in an async manner via the `async_query` method (uses an async httpx client). 

In [ ]:
results = await consdb_fastapi.async_query(query)
results.head(3)[['visit_id', 'obs_start', 's_ra', 's_dec', 'sky_rotation', 'band', 'target_name', 'observation_reason']]

There is also a convenience function to query all visit1 and visit1_quicklook data between two times. This is inherited from the parent class, for any of the Consdb (specific service) class above.  This only uses the synchronous client. 

In [ ]:
t_start = Time("2025-11-27T12:00:00", format='isot', scale='tai')
t_end = Time("2025-11-28T12:00:00", format='isot', scale='tai')

visits = consdb_tap.get_visits("lsstcam", t_start, t_end, visit_constraint="science_program = 'BLOCK-407'", augment=False)
print(len(visits))
visits.head()

## Incorrect SQL responses ##

Sending an incorrect sql query will result in an error message that includes the response from the ConsDb Postgres server, if you use the FastAPI interface, and a variety of sources for the error if using the TAP interface.

In [ ]:
from asyncio import sleep

# A bad query that attempts to access a non-existent table
bad_query = "select * from cdb_lsstcam.visits where science_program = 'BLOCK-407' and day_obs=20251127"
print(bad_query)
print("")

print("FastAPI error message")
await sleep(0.5)
visits = consdb_fastapi.query(bad_query)
print("")

print("TAP error message")
visits = consdb_tap.query(bad_query)

In [ ]:
# A bad query that uses a good table but bad sql -- string literals must be enclosed by single quotes
bad_query = "select * from cdb_lsstcam.visit1 where science_program = BLOCK-407 and day_obs=20251127"
print(bad_query)
print("") 

print("FastAPI error message")
await sleep(0.5)
visits = consdb_fastapi.query(bad_query)
print("")

print("TAP error message")
visits = consdb_tap.query(bad_query)

In [ ]:
# A bad query that uses a good table but bad sql -- == is for pandas, not sql.
bad_query = "select * from cdb_lsstcam.visit1 where science_program = 'BLOCK-407' and day_obs==20251127"
print(bad_query)
print("")

print("FastAPI error message")
await sleep(0.5)
visits = consdb_fastapi.query(bad_query)
print("")

print("TAP error message")
visits = consdb_tap.query(bad_query)

In [ ]:
# A query using wildcards -- a single % will work for TAP but FastAPI requires two %%
bad_query = "select * from cdb_lsstcam.visit1 where day_obs=20251127 and observation_reason like '%dd%'"
print(bad_query)

print("FastAPI error message")
await sleep(0.5)
visits = consdb_fastapi.query(bad_query)
print(len(visits))
print("")

print("TAP ok")
visits = consdb_tap.query(bad_query)
print(len(visits))

print("")
print("Repeat with double %%")
good_query = "select * from cdb_lsstcam.visit1 where day_obs=20251127 and observation_reason like '%%dd%%'"

print("FastAPI ok")
visits = consdb_fastapi.query(good_query)
print(len(visits))

print("TAP ok")
visits = consdb_tap.query(good_query)
print(len(visits))

## Add extra computed values 

There is a function to add some additional useful columns, such as the predicted zeropoint (which can be used to estimate cloud extinction) or moon separation, along with additional values useful to match the opsim schema.

In [ ]:
t_start = Time("2025-11-27T12:00:00", format='isot', scale='tai')
t_end = Time("2025-11-28T12:00:00", format='isot', scale='tai')

visits = consdb_tap.get_visits("lsstcam", t_start, t_end, visit_constraint="science_program = 'BLOCK-407'", augment=False)


visits_plus = augment_visits(visits)
# What columns are added?
set(visits_plus.columns) - set(visits.columns)

## Removing bad visits

DM maintains a repo at [excluded_visits](https://github.com/lsst-dm/excluded_visits) to track known bad visits, and you can download bad visits from this repo.

In [ ]:
bad_visit_ids = fetch_excluded_visits(instrument="lsstcam")
bad_visit_ids_night = [b for b in bad_visit_ids if b in visits.visit_id.values]
print("Bad visits in this night - ")
print(np.sort(bad_visit_ids_night))
better_visits = exclude_visits(visits=visits, bad_visit_ids=bad_visit_ids_night)
print(len(visits), len(better_visits))

In [ ]:
# Let's look at a different night, with some bad visits recorded
t_start_2 = Time("2025-11-22T12:00:00", format='isot', scale='tai')
t_end_2 = Time("2025-11-23T12:00:00", format='isot', scale='tai')

visits_2 = consdb_tap.get_visits("lsstcam", t_start_2, t_end_2, visit_constraint="science_program = 'BLOCK-407'")
better_visits_2 = exclude_visits(visits=visits_2, bad_visit_ids=bad_visit_ids)
len(visits_2), len(better_visits_2), len([b for b in bad_visit_ids if str(b).startswith('20251122')])

## Targets and Visits 

The `lsst.sal.Scheduler.logevent_target` and `lsst.sal.Scheduler.logevent_observation`, coupled with `lsst.sal.Scheduler.logevent_nextvisit` can provide additional useful information, including the salIndex of the script which generated the observation. These can be retrieve with `targets_and_visits`. 

In [ ]:
# This requires querying the EFD as well as the ConsDB
endpoints = get_clients(site=site, auth_token=token)
target_visits, cols, target_obs, nv, v = targets_and_visits(t_start, t_end, endpoints, queue_index=1)

In [ ]:
target_visits.head()[['time_target', 'time_nextvisit', 'time_observation', 'scriptSalIndex', 'groupId','obs_start', 'observation_reason',  'target_name', 'band']]